In [3]:
import os.path as op
import os

import inspect
import torch
import importlib
from torch import nn
from torch.nn import functional as F
import torch.optim.lr_scheduler as lrs

import pytorch_lightning as pl

from transformers import LlamaForCausalLM, LlamaTokenizer
import random
from pandas.core.frame import DataFrame

import numpy as np
from peft import get_peft_config, get_peft_model, get_peft_model_state_dict, LoraConfig, TaskType, PeftModel, MoeLoraConfig, MoeLoraModel


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
CUDA SETUP: CUDA runtime path found: /home/kongxy/miniconda3/envs/MOE4REC/lib/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 8.0
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /home/kongxy/miniconda3/envs/MOE4REC/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cuda118.so...


In [4]:

# 假设 user_embeds 和 gate_weights 是对应的张量
user_embeds = torch.randn(4, 1, 4096)  # 示例数据
gate_weights = torch.randn(4, 1, 4, 1)  # 示例数据


In [10]:
# 将user_embeds沿第一维度分割为数个向量
user_embeds_split = user_embeds.split(1, dim=0)
user_embed_vectors = [user_embed.squeeze() for user_embed in user_embeds_split]  # 去除第二个向量并去除多余的维度
gate_weights_squeezed = gate_weights.squeeze(-1)  # 去掉最后一个维度
gate_weights_split = gate_weights_squeezed.split(1, dim=0)  # 沿第一个维度分割
gate_weight_vectors = [x.squeeze() for x in gate_weights_split]  # 去除多余的维度
# 组合为字典
paired_dict = {f'gate_weight_{i+1}': gw.numpy() for i, gw in enumerate(gate_weight_vectors)}
paired_dict.update({f'user_embed_{i+1}': gw.numpy() for i, gw in enumerate(user_embed_vectors)})

In [14]:
print(paired_dict)

{'gate_weight_1': array([ 1.8035704 , -1.4251345 , -0.20917957,  1.1816493 ], dtype=float32), 'gate_weight_2': array([-1.130517  , -0.53926915, -0.25794864, -1.0742768 ], dtype=float32), 'gate_weight_3': array([ 0.38494626, -0.27768913, -0.06496254,  0.42176992], dtype=float32), 'gate_weight_4': array([-0.71604526, -0.8914298 , -0.7220739 ,  0.37865233], dtype=float32), 'user_embed_1': array([-1.4660963 , -1.6457783 , -0.26348984, ...,  0.47030622,
       -0.19276133,  1.2298163 ], dtype=float32), 'user_embed_2': array([-0.40329742,  0.41290927, -0.95601976, ..., -1.1843139 ,
       -1.3202466 , -0.5464423 ], dtype=float32), 'user_embed_3': array([ 1.463175  , -0.46543968,  0.04877191, ...,  0.07611048,
        0.20288888, -0.03222864], dtype=float32), 'user_embed_4': array([-1.6075574, -1.1339449,  0.6124862, ..., -1.1770983,  1.7049038,
        0.5721847], dtype=float32)}
